In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['font.size'] = 12
plt.rcParams['savefig.format'] = 'pdf'
sns.set_style('darkgrid')

In [2]:
import polars as pl
import networkx as nx
from tqdm import tqdm

pl.Config.set_streaming_chunk_size(10_000)


# df = pl.read_parquet('DOT_Traffic_Speeds_NBE.parquet')  # Eager execution
df = pl.scan_parquet('Chicago_Traffic_Tracker_-_Historical_Congestion_Estimates_by_Segment_-_2024-Current.parquet')  # Lazy execution

def get_lazy_shape(lf):
    rows = lf.select(pl.len()).collect()[0,0]
    cols = len(lf.collect_schema().names())
    return (rows, cols)

print(get_lazy_shape(df))
df.collect().head()

(73522936, 21)


TIME,SEGMENT_ID,SPEED,STREET,DIRECTION,FROM_STREET,TO_STREET,LENGTH,STREET_HEADING,BUS_COUNT,MESSAGE_COUNT,HOUR,DAY_OF_WEEK,MONTH,RECORD_ID,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,START_LOCATION,END_LOCATION
datetime[ns],i32,f32,cat,cat,str,str,f32,cat,i32,i32,u8,u8,u8,str,f32,f32,f32,f32,str,str
2025-05-14 12:30:49,1,25.0,"""55th""","""EB""","""Pulaski""","""Central Park""",0.5,"""W""",1,9,12,4,5,"""0001-202505141730""",41.793068,-87.72316,41.79314,-87.713608,"""POINT (-87.7231602513 41.79306…","""POINT (-87.7136071496 41.79314…"
2025-05-14 12:30:49,2,27.0,"""55th""","""EB""","""Central Park""","""Kedzie""",0.5,"""W""",1,9,12,4,5,"""0002-202505141730""",41.79314,-87.713608,41.793377,-87.703552,"""POINT (-87.7136071496 41.79314…","""POINT (-87.7035554211 41.79337…"
2025-05-14 12:30:49,3,-1.0,"""55th""","""EB""","""Kedzie""","""California""",0.5,"""W""",0,0,12,4,5,"""0003-202505141730""",41.793377,-87.703552,41.793552,-87.693802,"""POINT (-87.7035554211 41.79337…","""POINT (-87.693799302 41.793553…"
2025-05-14 12:30:49,4,-1.0,"""55th""","""EB""","""California""","""Western""",0.5,"""W""",0,0,12,4,5,"""0004-202505141730""",41.793552,-87.693802,41.793739,-87.684044,"""POINT (-87.693799302 41.793553…","""POINT (-87.6840431555 41.79373…"
2025-05-14 12:30:49,5,23.0,"""Garfield""","""EB""","""Western""","""Damen""",0.5,"""W""",1,7,12,4,5,"""0005-202505141730""",41.793739,-87.684044,41.793777,-87.674301,"""POINT (-87.6840431555 41.79373…","""POINT (-87.6743019717 41.79377…"


In [4]:
df_links = (
    df
    .sort('TIME', descending=True)
    .unique(subset=['SEGMENT_ID'], keep='first')
).collect()

len(df_links)

1047

In [14]:
line_coordinates_list = list(zip(
    list(zip(
        df_links['START_LATITUDE'].to_list(),
        df_links['START_LONGITUDE'].to_list(),
    )),
    list(zip(
        df_links['END_LATITUDE'].to_list(),
        df_links['END_LONGITUDE'].to_list(),
    ))
))

In [15]:
import folium

m = folium.Map(location=[41.793068, -87.72316])


i = 0
for line_coordinates in line_coordinates_list:
    line = folium.PolyLine(locations=line_coordinates, color='blue', weight=5, opacity=0.8)
    line.add_to(m)
    # if i > 145:
    #     break
    i += 1
m

Слишком решетчатая топология это плохо. Когда выкидываешь 1 ребро у тебя на самом деле куча альтернативных путей почти такой же длины. Может если мы говорим про пробки там ситуация получше (потому что понятно что каждое изменение по всей сети идет в каком-то смысле)

GPT взять OSM и сматчить имеющеся рёбра с картой. Для других рёбер на карте надо как-то аппроксимировать скорости